In [1]:
import hail as hl
import numpy as np

### Get HailTable of height for UKB white British subset (used for GCTA rg estimates)

In [2]:
gs_bucket = 'gs://nbaya/ldscsim/'
mt0 = hl.read_matrix_table(gs_bucket+'hm3.50_sim_h2_0.08.mt/')
ht0 = mt0.select_cols(mt0.nonsim_phen).cols()

Initializing Spark and Hail with default parameters...
Running on Apache Spark version 2.2.3
SparkUI available at http://10.128.0.37:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.11-5f8236706b33
LOGGING: writing to /home/hail/hail-20190314-1746-0.2.11-5f8236706b33.log
2019-03-14 17:46:54 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


In [3]:
ht1 = ht0.rename({'s':'IID','nonsim_phen':'y'})
ht1 = ht1.annotate(FID = '0')
ht1 = ht1.key_by(ht1.FID)
ht1 = ht1.select(ht1.IID,ht1.y)
ht1 = ht1.key_by(ht1.IID)

In [4]:
ids = hl.import_table('gs://nbaya/split/gcta/gcta_20k.grm.id',no_header=True) #GRM ids
ids = ids.rename({'f0':'FID','f1':'IID'})
ids = set(ids.IID.take(ids.count()))
ht2 = ht1.filter(hl.literal(ids).contains(ht1['IID']))

2019-03-14 17:21:44 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (type not specified)
  Loading column 'f1' as type 'str' (type not specified)



In [5]:
ht2.IID.show()

2019-03-14 17:22:26 Hail: INFO: Coerced sorted dataset
2019-03-14 17:22:28 Hail: INFO: Coerced sorted dataset
2019-03-14 17:22:30 Hail: INFO: Coerced sorted dataset


+-----------+
| IID       |
+-----------+
| str       |
+-----------+
| "1000844" |
| "1001013" |
| "1001278" |
| "1001384" |
| "1002023" |
| "1002057" |
| "1003314" |
| "1003347" |
| "1003529" |
| "1003878" |
+-----------+
showing top 10 rows



In [6]:
n = ht2.count()
for i in range(1,11):
    pi = [1]*int(n/2) + [0]*int(n/2)
    randstate = np.random.RandomState(i)
    randstate.shuffle(pi)
    ht = ht2.add_index()
    ht = ht.annotate(label = hl.literal(pi)[hl.int32(ht.idx)])
    ht = ht.annotate(y1 = hl.cond(ht.label==1, ht.y, hl.null('float')))
    ht = ht.annotate(y2 = hl.cond(ht.label==0, ht.y, hl.null('float')))
    ht = ht.drop(ht.idx,ht.label,ht.y)
    ht = ht.order_by(ht.y1)
    ht.show()
#     ht.export(f'gs://nbaya/split/gcta/gcta_20k.s{i}.phen')

2019-03-14 16:57:12 Hail: INFO: Coerced sorted dataset
2019-03-14 16:57:13 Hail: INFO: Coerced sorted dataset
2019-03-14 16:57:49 Hail: INFO: Coerced sorted dataset
2019-03-14 16:57:50 Hail: INFO: Coerced sorted dataset
2019-03-14 16:57:52 Hail: INFO: Coerced sorted dataset


+-----+-----------+-----------+---------+
| FID | IID       |        y1 |      y2 |
+-----+-----------+-----------+---------+
| str | str       |   float64 | float64 |
+-----+-----------+-----------+---------+
| "0" | "1611860" | -3.83e+00 |      NA |
| "0" | "3999820" | -3.80e+00 |      NA |
| "0" | "2161718" | -3.41e+00 |      NA |
| "0" | "5626326" | -3.38e+00 |      NA |
| "0" | "2683239" | -3.33e+00 |      NA |
| "0" | "2290186" | -3.30e+00 |      NA |
| "0" | "2482129" | -3.29e+00 |      NA |
| "0" | "3376535" | -3.24e+00 |      NA |
| "0" | "1290594" | -3.20e+00 |      NA |
| "0" | "2842188" | -3.12e+00 |      NA |
+-----+-----------+-----------+---------+
showing top 10 rows



2019-03-14 16:58:30 Hail: INFO: Coerced sorted dataset
2019-03-14 16:58:32 Hail: INFO: Coerced sorted dataset
2019-03-14 16:58:34 Hail: INFO: Coerced sorted dataset


+-----+-----------+-----------+---------+
| FID | IID       |        y1 |      y2 |
+-----+-----------+-----------+---------+
| str | str       |   float64 | float64 |
+-----+-----------+-----------+---------+
| "0" | "3999820" | -3.80e+00 |      NA |
| "0" | "5044464" | -3.67e+00 |      NA |
| "0" | "2113870" | -3.64e+00 |      NA |
| "0" | "1216856" | -3.53e+00 |      NA |
| "0" | "5431973" | -3.49e+00 |      NA |
| "0" | "5626326" | -3.38e+00 |      NA |
| "0" | "2683239" | -3.33e+00 |      NA |
| "0" | "5847550" | -3.29e+00 |      NA |
| "0" | "2482129" | -3.29e+00 |      NA |
| "0" | "1524138" | -3.25e+00 |      NA |
+-----+-----------+-----------+---------+
showing top 10 rows



2019-03-14 16:59:08 Hail: INFO: Coerced sorted dataset
2019-03-14 16:59:10 Hail: INFO: Coerced sorted dataset
2019-03-14 16:59:11 Hail: INFO: Coerced sorted dataset


...

2019-03-14 16:59:47 Hail: INFO: Coerced sorted dataset
2019-03-14 16:59:48 Hail: INFO: Coerced sorted dataset
2019-03-14 16:59:50 Hail: INFO: Coerced sorted dataset


+-----+-----------+-----------+---------+
| FID | IID       |        y1 |      y2 |
+-----+-----------+-----------+---------+
| str | str       |   float64 | float64 |
+-----+-----------+-----------+---------+
| "0" | "1611860" | -3.83e+00 |      NA |
| "0" | "1042750" | -3.76e+00 |      NA |
| "0" | "2113870" | -3.64e+00 |      NA |
| "0" | "1216856" | -3.53e+00 |      NA |
| "0" | "1155634" | -3.49e+00 |      NA |
| "0" | "5626326" | -3.38e+00 |      NA |
| "0" | "2290186" | -3.30e+00 |      NA |
| "0" | "5847550" | -3.29e+00 |      NA |
| "0" | "2482129" | -3.29e+00 |      NA |
| "0" | "1524138" | -3.25e+00 |      NA |
+-----+-----------+-----------+---------+
showing top 10 rows



2019-03-14 17:00:24 Hail: INFO: Coerced sorted dataset
2019-03-14 17:00:25 Hail: INFO: Coerced sorted dataset
2019-03-14 17:00:27 Hail: INFO: Coerced sorted dataset


+-----+-----------+-----------+---------+
| FID | IID       |        y1 |      y2 |
+-----+-----------+-----------+---------+
| str | str       |   float64 | float64 |
+-----+-----------+-----------+---------+
| "0" | "1611860" | -3.83e+00 |      NA |
| "0" | "3999820" | -3.80e+00 |      NA |
| "0" | "5626326" | -3.38e+00 |      NA |
| "0" | "2290186" | -3.30e+00 |      NA |
| "0" | "2482129" | -3.29e+00 |      NA |
| "0" | "1736350" | -3.24e+00 |      NA |
| "0" | "3376535" | -3.24e+00 |      NA |
| "0" | "5529174" | -3.08e+00 |      NA |
| "0" | "5795174" | -3.07e+00 |      NA |
| "0" | "2955572" | -3.01e+00 |      NA |
+-----+-----------+-----------+---------+
showing top 10 rows



KeyboardInterrupt: 

### Run GWAS using same splits as above

In [3]:
ids = hl.import_table('gs://nbaya/split/gcta/gcta_20k.grm.id',no_header=True) #GRM ids
ids = ids.rename({'f0':'FID','f1':'IID'})
ids = set(ids.IID.take(ids.count()))
mt1 = mt0.filter_cols(hl.literal(ids).contains(mt0.s))

2019-03-14 17:47:33 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (type not specified)
  Loading column 'f1' as type 'str' (type not specified)



In [4]:
def gwas(mt, x, y, cov_list=[], with_intercept=True, pass_through=[], path_to_save=None, 
         normalize_x=False, is_std_cov_list=False):
    '''Runs GWAS'''
    
    mt = mt._annotate_all(col_exprs={'__y':y},
                           entry_exprs={'__x':x})
    if normalize_x:
        mt = mt.annotate_rows(__gt_stats = hl.agg.stats(mt.__x))
        mt = mt.annotate_entries(__x= (mt.__x-mt.__gt_stats.mean)/mt.__gt_stats.stdev) 
        mt = mt.drop('__gt_stats')
    
    if is_std_cov_list:
        cov_list = ['isFemale','age','age_squared','age_isFemale',
                    'age_squared_isFemale']+['PC{:}'.format(i) for i in range(1, 21)]
        
    if str in list(map(lambda x: type(x),cov_list)):
        cov_list = list(map(lambda x: mt[x] if type(x) is str else x,cov_list))
        
    cov_list = ([1] if with_intercept else [])+cov_list
    
    pass_through = list(set(['rsid']+pass_through))
    print(f'variables to pass through: {pass_through}')

    gwas_ht = hl.linear_regression_rows(y=mt.__y,
                                        x=mt.__x,
                                        covariates=cov_list,
                                        pass_through = pass_through)
    
    gwas_ht = gwas_ht.annotate_globals(with_intercept = with_intercept)
        
    gwas_ht = gwas_ht.rename({'rsid':'SNP'}).key_by('SNP')
        
    gwas_ht = gwas_ht.select(Z = gwas_ht.t_stat,
                             N = gwas_ht.n)
    
    ss_template = hl.read_table('gs://nbaya/rg_sex/hm3.sumstats_template.ht') # sumstats template as a hail table
    ss_template = ss_template.key_by('SNP')
        
    ss = ss_template.annotate(Z = gwas_ht[ss_template.SNP].Z,
                              N = gwas_ht[ss_template.SNP].N)
    
    if path_to_save is not None:
        ss.export(path_to_save)
        
    return ss

In [7]:
n = mt1.count_cols()
for i in range(1,101):
    phen = hl.import_table(f'gs://nbaya/split/gcta/gcta_20k.s{i}.phen',types={'y1':hl.tfloat64,'y2':hl.tfloat64},key='IID')
    mt = mt1.annotate_cols(y1 = phen[mt1.s].y1,
                           y2 = phen[mt1.s].y2)
    gwas(mt=mt,x=mt.dosage,y=mt.y1,is_std_cov_list=True,path_to_save=f'gs://nbaya/split/gcta/20k_sumstats.y1.s{i}.tsv.bgz')
    gwas(mt=mt,x=mt.dosage,y=mt.y2,is_std_cov_list=True,path_to_save=f'gs://nbaya/split/gcta/20k_sumstats.y2.s{i}.tsv.bgz')

2019-03-14 17:57:02 Hail: INFO: Coerced sorted dataset
2019-03-14 17:57:03 Hail: INFO: Reading table with no type imputation
  Loading column 'FID' as type 'str' (type not specified)
  Loading column 'IID' as type 'str' (type not specified)
  Loading column 'y1' as type 'float64' (user-specified)
  Loading column 'y2' as type 'float64' (user-specified)



variables to pass through: ['rsid']


2019-03-14 17:57:45 Hail: WARN: 10000 of 20000 samples have a missing phenotype or covariate.
2019-03-14 17:57:45 Hail: INFO: linear_regression_rows: running on 10000 samples for 1 response variable y,
    with input variable x, and 26 additional covariates...
2019-03-14 17:57:48 Hail: INFO: Coerced sorted dataset
2019-03-14 17:59:31 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-14 17:59:34 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-14 18:01:09 Hail: INFO: while writing:
    gs://nbaya/split/gcta/20k_sumstats.y1.s1.tsv.bgz
  merge time: 1.324s


variables to pass through: ['rsid']


2019-03-14 18:01:52 Hail: WARN: 10000 of 20000 samples have a missing phenotype or covariate.
2019-03-14 18:01:52 Hail: INFO: linear_regression_rows: running on 10000 samples for 1 response variable y,
    with input variable x, and 26 additional covariates...
2019-03-14 18:01:55 Hail: INFO: Coerced sorted dataset
2019-03-14 18:03:37 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-14 18:03:37 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-14 18:05:05 Hail: INFO: while writing:
    gs://nbaya/split/gcta/20k_sumstats.y2.s1.tsv.bgz
  merge time: 1.037s
2019-03-14 18:05:06 Hail: INFO: Reading table with no type imputation
  Loading column 'FID' as type 'str' (type not specified)
  Loading column 'IID' as type 'str' (type not specified)
  Loading column 'y1' as type 'float64' (user-specified)
  Loading column 'y2' as type 'float64' (user-specified)



...

variables to pass through: ['rsid']


2019-03-14 18:22:26 Hail: WARN: 10000 of 20000 samples have a missing phenotype or covariate.
2019-03-14 18:22:26 Hail: INFO: linear_regression_rows: running on 10000 samples for 1 response variable y,
    with input variable x, and 26 additional covariates...
2019-03-14 18:22:28 Hail: INFO: Coerced sorted dataset
2019-03-14 18:24:09 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-14 18:24:09 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-14 18:25:40 Hail: INFO: while writing:
    gs://nbaya/split/gcta/20k_sumstats.y1.s4.tsv.bgz
  merge time: 1.262s


variables to pass through: ['rsid']


2019-03-14 18:26:21 Hail: WARN: 10000 of 20000 samples have a missing phenotype or covariate.
2019-03-14 18:26:21 Hail: INFO: linear_regression_rows: running on 10000 samples for 1 response variable y,
    with input variable x, and 26 additional covariates...
2019-03-14 18:26:24 Hail: INFO: Coerced sorted dataset
2019-03-14 18:28:07 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-14 18:28:29 Hail: INFO: Ordering unsorted dataset with network shuffle


FatalError: SparkException: Job aborted due to stage failure: ResultStage 85 (saveAsTextFile at RichRDD.scala:66) has failed the maximum allowable number of times: 4. Most recent failure reason: org.apache.spark.shuffle.FetchFailedException: Failure while fetching StreamChunkId{streamId=397532422004, chunkIndex=0}: java.lang.RuntimeException: Executor is not registered (appId=application_1552581400009_0003, execId=53) 	at org.apache.spark.network.shuffle.ExternalShuffleBlockResolver.getBlockData(ExternalShuffleBlockResolver.java:171) 	at org.apache.spark.network.shuffle.ExternalShuffleBlockHandler$1.next(ExternalShuffleBlockHandler.java:105) 	at org.apache.spark.network.shuffle.ExternalShuffleBlockHandler$1.next(ExternalShuffleBlockHandler.java:95) 	at org.apache.spark.network.server.OneForOneStreamManager.getChunk(OneForOneStreamManager.java:87) 	at org.apache.spark.network.server.TransportRequestHandler.processFetchRequest(TransportRequestHandler.java:125) 	at org.apache.spark.networ